In [1]:
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from arch import arch_model
import fix_yahoo_finance as yf
from pandas_datareader import data as pdr


In [2]:
date_start = "2010-01-01"
date_end = "2017-12-31"

sym1 = 'MSFT' 

data = pd.DataFrame()
yf.pdr_override() 
data = pdr.get_data_yahoo(sym1, start=date_start, end=date_end)
data['returns'] = np.log(data['Adj Close'] / data['Adj Close'].shift(1))
data = data.dropna() 
data['sqrReturns'] = data.returns**2

hvar = np.mean(data.sqrReturns)
hstd = np.sqrt(hvar) #historic daily vol

# Fit a GARCH(1, 1) model 
# We use the arch_model function from the ARCH package

am = arch_model(data.returns, vol='Garch', p=1, o=0, q=1, dist='Normal')
res = am.fit(update_freq=5)
print(res.summary())

condvol=res.conditional_volatility.iloc[-1]
print('Annualized Volatility - Historic Method: %5.2f%%' % (hstd*np.sqrt(252)*100))
print('Annualized Volatility - Garch(1,1): %5.2f%%' % (condvol*np.sqrt(252)*100))


forecasts = res.forecast(horizon=5)
print(forecasts.variance.tail())

[*********************100%***********************]  1 of 1 downloaded
Iteration:      5,   Func. Count:     47,   Neg. LLF: -5762.344992495958
Iteration:     10,   Func. Count:     85,   Neg. LLF: -5763.501742672557
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -5763.515898452715
            Iterations: 12
            Function evaluations: 110
            Gradient evaluations: 12
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                      GARCH   Log-Likelihood:                5763.52
Distribution:                  Normal   AIC:                          -11519.0
Method:            Maximum Likelihood   BIC:                          -11496.6
                                        No. Observations:                 2013
Date:              

In [3]:
import fintech.finmodels as fm

In [7]:
conditional_vol = fm.garch_vol(data.returns)
annualized_vol = (conditional_vol*np.sqrt(252)*100)
print("Annualized Volatility - Garch(1,1): {0}%".format(round(annualized_vol,2)))

Iteration:      5,   Func. Count:     47,   Neg. LLF: -5762.344992495958
Iteration:     10,   Func. Count:     85,   Neg. LLF: -5763.501742672557
Optimization terminated successfully.    (Exit mode 0)
            Current function value: -5763.515898452715
            Iterations: 12
            Function evaluations: 110
            Gradient evaluations: 12
                     Constant Mean - GARCH Model Results                      
Dep. Variable:                returns   R-squared:                      -0.000
Mean Model:             Constant Mean   Adj. R-squared:                 -0.000
Vol Model:                      GARCH   Log-Likelihood:                5763.52
Distribution:                  Normal   AIC:                          -11519.0
Method:            Maximum Likelihood   BIC:                          -11496.6
                                        No. Observations:                 2013
Date:                Fri, Feb 01 2019   Df Residuals:                     2009
Time:     